In [7]:
from quantize import quantize
import numpy as np

f = lambda x: x
w = np.array(range(10))
print(w.shape)
w = quantize(w,2, f)
w

(10,)


array([2.25 , 2.25 , 2.25 , 3.375, 3.375, 5.625, 5.625, 6.75 , 6.75 ,
       6.75 ])